# Create the CSV File Mapping NFTs to their Rank
---
This uses the same steps and functions as the NFT rank calculation in `calculations.ipynb`. However, in the `calculations.ipynb` file, we illustrate the process and then pull from a CSV file for the rankings. In this notebook, we will use those steps to create the CSV file needed instead.

---
## 1.1 Open Statements

In [1]:
open System.Text.Json
open System.IO
open FSharp.Reflection

## 1.2 Calculation Steps
For a detailed explanation of this process, please refer to `calculations.ipynb`. However, if you want to create the CSV file, run all the cells in this notebook. However, please first make sure that you have the JSON files from running `webscraper.ipynb` and `attributecounter.ipynb`.

#### Fetching Data

In [2]:
type NFT = {
    assetName: string
    encodedName: string
    Background: string
    Bear: string
    Clothes: string
    Face: string
    Handheld: string
    Head: string
    Skins: string
}

type NFTTraits = {
    Background: Map<string, int>
    Bear: Map<string, int>
    Clothes: Map<string, int>
    Face: Map<string, int>
    Handheld: Map<string, int>
    Head: Map<string, int>
    Skins: Map<string, int>
}

In [3]:
let loadNFTs<'T> (filePath: string) : 'T =
    let jsonContent = File.ReadAllText(filePath)
    JsonSerializer.Deserialize<'T>(jsonContent)

In [ ]:
let nftRound1NFTS = loadNFTs<NFT list> "../data/collection_1.json"
let nftRound2NFTS = loadNFTs<NFT list> "../data/collection_2.json"

In [ ]:
let nftRound1Traits = loadNFTs<NFTTraits> "../data/collectiontraits_1.json"
let nftRound2Traits = loadNFTs<NFTTraits> "../data/collectiontraits_2.json"

#### Calculating Trait Value Scores

In [6]:
let r1TotalNfts = 804.0
let r2TotalNfts = 2035.0

In [7]:
let processNFTTraits (nftTraits: NFTTraits) (totalNfts: float) =
    let recordType = typeof<NFTTraits>
    FSharp.Reflection.FSharpType.GetRecordFields(recordType)
    |> Array.fold (fun acc field ->
        let fieldValue = field.GetValue(nftTraits) :?> Map<string, int>
        let traitScores = 
            fieldValue |> Map.map (fun key value ->
                Math.Round(100.0 - (float value / float totalNfts) * 100.0, 2)
            )
        Map.add field.Name traitScores acc
    ) Map.empty

In [8]:
let r1ScoredTraits = processNFTTraits nftRound1Traits r1TotalNfts
let r2ScoredTraits = processNFTTraits nftRound2Traits r2TotalNfts

#### Compute NFT Ranks

In [9]:
let computeNFTRank (nftList: list<NFT>) nftTraitScores = 
    nftList
    |> List.map (fun nft ->
        let rarityScore =  
            [
                ("Background", nft.Background)
                ("Bear", nft.Bear)
                ("Clothes", nft.Clothes)
                ("Face", nft.Face)
                ("Handheld", nft.Handheld)
                ("Head", nft.Head)
                ("Skins", nft.Skins)
            ]
            |> List.fold (fun acc (attrName, attrValue) ->  
                match Map.tryFind attrName nftTraitScores with
                | Some traitMap ->  
                    acc + (Map.tryFind attrValue traitMap |> Option.defaultValue 0.0)
                | None -> acc 
            ) 0.0 
        (nft.assetName, nft.encodedName, rarityScore) 
    )
    |> List.sortBy (fun (_, _, rarityScore) -> -rarityScore)
    |> List.mapi (fun index (assetName, encodedName, rarityScore) -> 
    (index + 1, assetName, encodedName, rarityScore))

In [10]:
let rankedNFTround1 = computeNFTRank nftRound1NFTS r1ScoredTraits
let rankedNFTround2 = computeNFTRank nftRound2NFTS r2ScoredTraits

#### CSV for Subject to Rank
This section creates the CSV file mapping a subject to its rank.

In [11]:
let listToCsv (data: (int * string * string * float) list) (policyId: string) (filePath: string) =
    let isNewFile = not (File.Exists(filePath))
    
    let header = "Name,Subject,Rank,Score"
    let rows =
        data
        |> List.map (fun (rank, assetName, encodedName, score) ->  
            let subject = sprintf "%s%s" policyId encodedName  
            sprintf "%s,%s,%d,%f" assetName subject rank score
        )
    
    let existingRows = 
        if not isNewFile then
            File.ReadAllLines(filePath) |> Set.ofSeq
        else
            Set.empty

    let newRows = 
        rows
        |> List.filter (fun row -> not (existingRows.Contains row))
    
    if isNewFile then
        File.AppendAllLines(filePath, [header])

    if newRows.Length > 0 then
        File.AppendAllLines(filePath, newRows)

In [ ]:
let policyIdRound1 = "ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d87";
let policyIdRound2 = "da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c";

listToCsv rankedNFTround1  policyIdRound1 "../data/collectionranks.csv"
listToCsv rankedNFTround2 policyIdRound2 "../data/collectionranks.csv"